# Final Project 

In [1]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import animatplot as amp

from itertools import chain
from numpy import pi, cos, sin, exp, sqrt
from scipy.signal import freqz, welch, periodogram, butter, lfilter, filtfilt, boxcar, ricker, cwt, ellip
from scipy.interpolate import griddata
from scipy.spatial import Voronoi, voronoi_plot_2d

from matplotlib.dates import DateFormatter, MinuteLocator, HourLocator, MonthLocator
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from matplotlib.ticker import FixedFormatter

from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'
# %matplotlib notebook

## Import and clean up the data

In [2]:
# Import data
# All data (hour resolution)
AS = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=[0, 1])

In [3]:
# Individual stations (minuite resolution)
DC = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DeepCove_temperature.dat', 
                 header=2)

DE = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DiscoveryElementary_temperature.dat', 
                 header=2)

HL = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/Helgesen_temperature.dat', 
                 header=2)

JB = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JamesBay_temperature.dat', 
                 header=2)

JM = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JohnMuir_temperature.dat', 
                 header=2)

KT = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/Keating_temperature.dat', 
                 header=2)

US = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/UVicSci_temperature.dat', 
                 header=2)

# Coastline 
MAP = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DataCoast.dat', 
                 sep='\s+', names = ["Lon", "Lat"])

### Clean up the All Stations data

In [4]:
# Convert times from MatLab time to Python Time
AS['Time'] = AS['NaN', 'NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
AS2 = AS.rename(index=str, columns={"NaN": "MatLab Time"})

# Set time as index column
AS3 = AS2.set_index('Time')

# Get the longitudes correct 
for i in range(1, np.shape(AS3)[1]):
    New_name = float(list(AS3)[i][1]) - 360
    AS3 = AS3.rename(columns={list(AS)[i][1]: str(format(New_name, '.4f'))})

AS4 = AS3.drop('MatLab Time', axis=1, level=1)
    
AS = AS4

In [5]:
# De-Nan
AS_DN = AS.dropna(axis=0)

### Clean up the minute resolution data

In [6]:
def DateInsert(Data):
    date = pd.date_range(start='2011-12-31 17:00:00.000000', 
                     freq='min', periods = len(Data))
    # Insert dates into D2 dataframe
    Data.insert(loc=0, column='Time', value=date)
    # Rename the columns
    D1 = Data.rename(index=str, columns={Data.columns[1]: "Temperature"})
    # Set index
    DM = D1.set_index('Time')
    return DM

In [7]:
DataM = [DC, DE, HL, JB, JM, KT, US]
DataMLabels = ['Deep Cove', 'Discovery', 'Helgesen', 'James Bay', 'John Muir', 'Keating', 'UVicSci']

In [8]:
DataMIns = [DateInsert(DataM[i]) for i in range(0, len(DataM))]

In [9]:
Mn = {
    "DC" : DataMIns[0].rename(columns={"Temperature": "DC"}),
    "DE" : DataMIns[1].rename(columns={"Temperature": "DE"}),
    "HL" : DataMIns[2].rename(columns={"Temperature": "HL"}),
    "JB" : DataMIns[3].rename(columns={"Temperature": "JB"}),
    "JM" : DataMIns[4].rename(columns={"Temperature": "JM"}),
    "KT" : DataMIns[5].rename(columns={"Temperature": "KT"}),
    "US" : DataMIns[6].rename(columns={"Temperature": "US"})
}

In [10]:
# De-Nan
Mn_DN  = {
    "DC" : Mn["DC"].dropna(axis=0),
    "DE" : Mn["DE"].dropna(axis=0),
    "HL" : Mn["HL"].dropna(axis=0),
    "JB" : Mn["JB"].dropna(axis=0),
    "JM" : Mn["JM"].dropna(axis=0),
    "KT" : Mn["KT"].dropna(axis=0),
    "US" : Mn["US"].dropna(axis=0)
}

In [11]:
# Matrix of all the minute resolution data
MS = pd.concat([Mn['DC'], Mn['DE'], Mn['HL'], Mn['JB'], Mn['JM'], Mn['KT'], Mn['US']], axis=1, sort=True)

### Get station locations

In [12]:
# Station locations
StationLoc = list(AS)
Lon = np.array([float(list(AS)[i][1]) for i in range(1, np.shape(list(AS))[0])])
Lat = np.array([float(list(AS)[i][0]) for i in range(1, np.shape(list(AS))[0])])

## Create meshgrid for interpolation

In [13]:
step_size = 0.001
x = np.arange(np.min(MAP['Lon']), np.max(MAP['Lon']), step_size)
y = np.arange(np.min(MAP['Lat']), np.max(MAP['Lat']), step_size)

X, Y = np.meshgrid(x, y)

# Get the locations of the data
AS_loc = np.array([[float(AS.columns.values[i][1]), float(AS.columns.values[i][0])] for i in range(len(AS.columns.values))])

## Useful functions

In [14]:
# Get dimensions 
def get_dim(x=X, y=Y):
    dim = [np.min(x), np.max(x), np.min(y), np.max(y)]
    Dim = [float(dim[n]) for n in range(len(dim))]
    return Dim

In [15]:
# Draw the map
def map(title='WHAT\'S THE THE TITLE YOU DONKEY?!?!?', size=(10, 10), land='#FFFFFF', water='#FFFFFF'):
    fig, ax = plt.subplots(1, 1, figsize=size)
    
    AddPoints = pd.DataFrame([[MAP['Lon'].min()-0.1, MAP['Lat'].min()-0.1], [MAP['Lon'].max()+0.1, MAP['Lat'].min()-0.1], [MAP['Lon'].max()+0.1, MAP['Lat'].max()+0.1]], columns=['Lon', 'Lat'])
    MAPIn = MAP.append(AddPoints, ignore_index=True)
    
    ax.fill(MAP['Lon'], MAP['Lat'], land, zorder=0, label='')
    ax.fill(MAPIn['Lon'], MAPIn['Lat'], water, zorder=2, label='')
    ax.plot(MAP['Lon'], MAP['Lat'], linewidth=np.min(size)/12, color='k', zorder=3, label='')

    ax.xaxis.set_major_formatter(StrMethodFormatter(r'{x:.1f}$^\circ$W'))
    ax.yaxis.set_major_formatter(StrMethodFormatter(r'{x:.2f}$^\circ$N'))
    mp.rc('xtick', labelsize=np.min(size)*1.7) 
    mp.rc('ytick', labelsize=np.min(size)*1.7) 
    plt.xticks(rotation=0)
    
    # Old and depricated string format:
    # Map2.xaxis.set_major_formatter(FormatStrFormatter(r'%1.1f$^\circ$W'))
    # Map2.yaxis.set_major_formatter(FormatStrFormatter(r'%1.2f$^\circ$N'))

    ax.set_xlim(-123.8, -123.25)
    ax.set_ylim(48.325, 48.725)
#     ax.grid(dashes=(1,1), color='#555555', zorder=0, linewidth=size[0]/6)
    ax.set_facecolor(water)
    
    ax.set_aspect('equal')
    maptitle = title
    ax.set_title("\n".join(wrap(maptitle, 50)), fontsize=np.min(size)*2.25)
    ax.set_xlabel(r'Longitude', fontsize=np.min(size)*2)
    ax.set_ylabel(r'Latitude', fontsize=np.min(size)*2)

In [16]:
# Heatmap for...um...map?
def heatmap(Array, D=get_dim(), colourmap='coolwarm', size=(10, 8), title='TITLE DONKEY!!!', 
            units=r'Temperature difference from mean [$^\circ$C]', interp='none'):
    
    map(title, size, land='#FFFFFF', water='#EEEEEE')
    plt.scatter(Lon, Lat, marker='o', s=5, color='k', zorder=1)
    Array_plot = plt.imshow(Array, extent=D, origin='lower', cmap=colourmap, interpolation=interp)
    cb = plt.colorbar(Array_plot, shrink=0.7, aspect=20, format='%.3f', label=units)
    cb.set_label(label=units, fontsize=np.min(size)*2)

In [17]:
# Heatmap for time series
def heatmapH(Array=AS, size=(20, 15), colour='coolwarm', xscale=1/5, yscale=1.035, 
            title='TITLE YA DONKEY!!', yLabel='WHAT IS IT YA DONKEY?!?'):
    
    fig, ax = plt.subplots(1, 1, figsize=size)
    hm = ax.imshow(np.array(Array.T), 
                   extent=(0, size[0]*10, 0, len(Array.columns.values)*size[1]*xscale), 
                   origin='lower', cmap=colour, interpolation='nearest', filternorm=False)

    cb = plt.colorbar(hm, shrink=0.5, aspect=20, format='%.3f', label=r'$^\circ$C')
    cb.set_label(label=r'Temperature [$^\circ$C]', fontsize=np.min(size)*1.5)

    # Beautify xticks
    t_range = pd.date_range(Array.index[0], Array.index[-1], freq='6M')
    xticks = ax.get_xticks()
    Xticks = np.arange(0, np.max(xticks), np.max(xticks)/len(t_range) * yscale)
    xformat = [i.strftime('%b-%Y') for i in t_range]
    plt.xticks(Xticks, xformat, rotation=90)

    # Beautify yticks
    plt.yticks(np.arange(0.5*3*5*xscale, (len(Array.columns.values)+0.5)*3*5*xscale, 3*5*xscale), np.array(Array.columns.values), fontsize=np.min(size))
    # plt.xticks(AS.index.resample('Y'))
    # plt.xticks(np.arange(0, 200, int(200/len(Dates))), Dates, fontsize=10) # len(Dates) can not be too long, otherwise divide by zero error
    ax.set_facecolor('k')
    ax.set_title(title, fontsize=np.min(size)*2)
    ax.set_xlabel('Dates', fontsize=np.min(size)*1.5)
    ax.set_ylabel(yLabel, fontsize=np.min(size)*1.5)

In [18]:
# Emperical orthogonal functions
def EOF(Array=AS_DN, int_type='cubic', locations=AS_loc):
    # The covariance matrix
    mean = Array.mean(axis=1)
#     Array_DM = Array.sub(Array.mean(axis=1), axis=0)
#     Array2 = Array.interpolate(method='cubic', axis=0)
    Array_DM = Array - Array.mean()
    Array_DN = Array_DM.fillna(0)
    Array_CM = Array_DN.cov()

    # Eigenvalue and Eigenvector matrix
    Array_CM_Eval, Array_CM_Evec = np.linalg.eig(Array_CM)
    Array_CM_EvalM = np.identity(len(Array_CM_Eval)) * Array_CM_Eval
    
    # Normalize them
#     Array_CM_Evec = Array_CM_Evec/sum(Array_CM_Evec)
    Array_CM_Eval = Array_CM_Eval/sum(Array_CM_Eval)
    
    # Get the EOFs
    EOFs = np.array([griddata(locations, Array_CM_Evec[:,i], (X, Y), method=int_type) for i in range(len(Array_CM_Eval))])
    return EOFs, Array_CM_Evec, Array_CM_Eval

In [19]:
def plot_power(Array, column=0 ,seg=2 , con_int=0.95, fs=1/60, size=(10, 4), Wtype='spectrum',
               title="TITLE DONKEY!!!", unit='Hz', Unit=1):  
    
    # De-Nan
    Array2 = Array[Array.columns[column]]
    Array_DN = Array2[~np.isnan(Array2)]

    # Lengths 
    Array_DN_L = len(Array_DN)/(2**seg)

    # Now calculate
    F, G = welch(Array_DN, fs, nperseg=Array_DN_L,\
                        window=sci.signal.windows.hann(int(Array_DN_L)),\
                        noverlap = Array_DN_L/2, nfft = Array_DN_L, detrend=False,\
                        return_onesided=True, scaling = Wtype)

    # Plot the power spectra
    fig, Array_PS = plt.subplots(1, 1, figsize=size)
    Array_PS.loglog(F*Unit/(2*pi), G, label = r'PDS of $x_f(t)$', linewidth=1)
    
    # Plot the uncertainties 
    dof = 2*np.floor(len(Array)/Array_DN_L)

    inter= sci.stats.chi2.interval(con_int, int(dof))
    Gmin = G*dof/inter[1]
    Gmax = G*dof/inter[0]

    Array_PS.fill_between(F*Unit/(2*pi), Gmin, Gmax, alpha=0.5, label='{0:.2f} Confidence Interval'.format(con_int*100))

    Array_PS.set_title(title)
    Array_PS.set_xlabel("Frequency [{0}]".format(unit))

    Array_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')

    fig.tight_layout()
    plt.show()

In [20]:
# Filter functions
# =================

# High/Low-pass filters 
def Ellip(Cut, rp, rs, fs, order, btype):
    fn = fs/2
    Ws = Cut / fn
    b, a = ellip(order, rp, rs, Ws, btype, output='ba')
    return b, a

def EllipF(x, Cut, rp, rs, fs, order, btype):
    b, a = Ellip(Cut, rp, rs, fs, order, btype)
    y = filtfilt(b, a, x)
    return y

def GraphEllip(Cut, rp, rs, fs, order, btype, worN=2*10**5):
    b, a = Ellip(Cut, rp, rs, fs, order, btype)
    w, h = freqz(b, a, worN)
    return b, a, w, h

# Bandpass filters 
def EllipBP(CutL, CutH, rp, rs, fs, order, btype):
    fn = fs/2
    low = CutL / fn
    high = CutH / fn
    b, a = ellip(order, rp, rs, [low, high], btype, output='ba')
    return b, a

def EllipFBP(x, CutL, CutH, rp, rs, fs, order, btype):
    b, a = EllipBP(CutL, CutH, rp, rs, fs, order, btype)
    y = filtfilt(b, a, x)
    return y

def GraphEllipBP(CutL, CutH, rp, rs, fs, order, btype, worN=2*10**5):
    b, a = EllipBP(CutL, CutH, rp, rs, fs, order, btype)
    w, h = freqz(b, a, worN)
    return b, a, w, h

In [21]:
def plot_ellip_diurnal(FS=1/60):
    # Filters 
    [CutH, rpH, rsH, orderH, btypeH] = [1/(24*3600*2*pi), 2, 50, 4, 'highpass']
    [CutL, rpL, rsL, orderL, btypeL] = [1/(24*3600*2*pi), 2, 50, 4, 'lowpass']

    bH, aH, wH, hH = GraphEllip(Cut=CutH, rp=rpH, rs=rsH, fs=FS, order=orderH, btype=btypeH)
    bL, aL, wL, hL = GraphEllip(Cut=CutL, rp=rpL, rs=rsL, fs=FS, order=orderL, btype=btypeL)

    fig, ellip_plot = plt.subplots(1, 1, figsize=(6, 4))

    ellip_plot.plot(FSm * wH/(2*pi), abs(hH), label="Type: {0}, Order: {1}, Cut-off: {2:.2e} rad/s".format(btypeH, orderH, CutH))
    ellip_plot.plot(FSm * wL/(2*pi), abs(hL), label="Type: {0}, Order: {1}, Cut-off: {2:.2e} rad/s".format(btypeL, orderL, CutL))
    ellip_plot.plot(FSm * wL/(2*pi), abs(hL*DCs_hH)/np.max(hL*hH), label="Type: {0}, Order: {1}, Cut-off: {2:.2e} rad/s".format(btypeL, orderL, CutL))
    ellip_plot.axvline(1/(24*3600*2*pi), color='k', linestyle='--', label=('Diurnal Frequency'))
    ellip_plot.set_title('Elliptic filter used to isolate the diurnal frequency')
    ellip_plot.set_xlabel(r'Frequency ($Hz$)')
    ellip_plot.set_ylabel(r'$H(\omega)$')
    ellip_plot.grid(True)
    ellip_plot.legend(loc=3)
    ellip_plot.set_xlim(1.5*10**(-6), 2.2*10**(-6))
    ellip_plot.set_ylim(10**-1, 1.15)
    ellip_plot.yaxis.set_major_formatter(FormatStrFormatter('%.1e'))
    ellip_plot.xaxis.set_major_formatter(FormatStrFormatter('%.1e'))
    ellip_plot.tick_params(axis='both', which='major', labelsize=8)
#     ellip_plot.tick_params(axis='both', which='minor', labelsize=8)
    plt.show()

In [22]:
def plot_ellip_T(Array, column=0, Cut=24*3600, FS=1/60, size=(10, 4), Wtype='spectrum',
                title="Title YA DONKEY!!!"):
    # De-Nan
#     Array2 = Array[Array.columns[column]]
    Array2 = Array
    Array_DN = Array2[~np.isnan(Array2)]
    
#     dr = pd.date_range(Array_DN.index[0], Array_DN.index[-1], freq='m')
    t = Array_DN.index.values

    # Lengths 
#     Array_DN_L = len(Array_DN)/(2**seg)

    # Now calculate
#     F, G = welch(Array_DN, fs, nperseg=Array_DN_L,\
#                         window=sci.signal.windows.hann(int(Array_DN_L)),\
#                         noverlap = Array_DN_L/2, nfft = Array_DN_L, detrend=False,\
#                         return_onesided=True, scaling = Wtype)
    
    # Filters 
    [CutH, rpH, rsH, orderH, btypeH] = [1/(Cut*2*pi), 2, 50, 4, 'highpass']
    [CutL, rpL, rsL, orderL, btypeL] = [1/(Cut*2*pi), 2, 50, 4, 'lowpass']

    bH, aH, wH, hH = GraphEllip(Cut=CutH, rp=rpH, rs=rsH, fs=FS, order=orderH, btype=btypeH)
    bL, aL, wL, hL = GraphEllip(Cut=CutL, rp=rpL, rs=rsL, fs=FS, order=orderL, btype=btypeL)
    
    # Filter the data
    FH = EllipF(Array_DN, Cut=CutH, rp=rpH, rs=rsH, fs=FS/(2*pi), order=orderH, btype=btypeH)
    FHL = EllipF(FH, Cut=CutL, rp=rpL, rs=rsL, fs=FS/(2*pi), order=orderL, btype=btypeL)
    
    # Rescale the data
    FHS = FHL/np.max(hL*hH)

    fig, ellip_plot = plt.subplots(1, 1, figsize=(10, 4))

    ellip_plot.plot(t, Array, label='Original data')
    ellip_plot.plot(t, FHS, label='Dirunal frequency')
    ellip_plot.set_title(title)
    ellip_plot.set_xlabel(r'Time')
    ellip_plot.set_ylabel(r'Temperature $^\circ$C')
    ellip_plot.grid(True)
    ellip_plot.legend()
    ellip_plot.tick_params(axis='both', which='major', labelsize=10, rotation=90)
    return t, FHS

In [23]:
# plot_ellip(Mn['DC'])
Array = MS["DC"].loc['2015-06-01 00:00':'2015-09-30 23:59'].resample('h').mean()
plot_ellip_T(Array, FS=1/3600)
# Array2 = Array
# Array_DN = Array2[~np.isnan(Array2)]
    
# # dr = pd.date_range(Array_DN.index[0], Array_DN.index[-1], freq='m')
# t = Array.index.values

/Users/Kev/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/Users/Kev/anaconda3/lib/python3.7/site-packages/numpy/core/numeric.py:501: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


(array(['2015-06-01T00:00:00.000000000', '2015-06-01T01:00:00.000000000',
        '2015-06-01T02:00:00.000000000', ...,
        '2015-09-30T21:00:00.000000000', '2015-09-30T22:00:00.000000000',
        '2015-09-30T23:00:00.000000000'], dtype='datetime64[ns]'),
 array([-0.3889896 -0.00133197j, -0.6825748 -0.00233725j,
        -0.91303415-0.00312639j, ...,  0.2417364 +0.00082775j,
         0.04986773+0.00017076j, -0.15426   -0.00052821j]))

<Figure size 720x288 with 1 Axes>

In [24]:
# # dr
# plt.figure()
# plt.plot(t, Array)
# np.shape(t), np.shape(Array)
# DataMIns[0].resample('h').mean()

## Plot the heatmap for all stations data

In [25]:
heatmapH(title='Temperature values of each station (hour resolution)', yLabel=r'Station locations $[^\circ N, ^\circ W]$')

<Figure size 1440x1080 with 2 Axes>

In [26]:
# Drop the bad ones
HS = AS.drop(columns=['48.4359', '48.4623', '48.6568', '48.4456', '48.4161', '48.3874', '48.5745', '48.4562', '48.6701'])
# HS = AS.DataFrame.drop([1, 2, 6, 10, 13], axis=0)
heatmapH(Array=HS, title='Temperature values of each station (hour resolution)', 
         yLabel=r'Station locations $[^\circ N, ^\circ W]$')\

/Users/Kev/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


<Figure size 1440x1080 with 2 Axes>

In [27]:
# Interpolate the hour resolution data
HSi = HS.interpolate(method='linear', axis=0)
heatmapH(Array=HSi, title='Temperature values of each station (hour resolution)', 
         yLabel=r'Station locations $[^\circ N, ^\circ W]$')\

<Figure size 1440x1080 with 2 Axes>

In [28]:
# Heatmap for the minute resolution 
heatmapH(Array=MS, xscale=0.75, size=(20, 15),
         title='Temperature values of each station (minute resolution)',
         yLabel=r'Station locations $[^\circ N, ^\circ W]$')

<Figure size 1440x1080 with 2 Axes>

In [29]:
# Interpolate the minute resolution data
MSi = MS.interpolate(method='linear', axis=0)
heatmapH(Array=MSi, xscale=0.75, size=(20, 15),
         title='Temperature values of each station (minute resolution)',
         yLabel=r'Station locations $[^\circ N, ^\circ W]$')

<Figure size 1440x1080 with 2 Axes>

## Plot the map with base stations 

In [30]:
# Plot the base stations
map2title = 'Location of weather temperature measurement stations in south-east Vancouver Island'
map(map2title, land='#CCEECC', water='#DDEEFF')
plt.scatter(Lon, Lat, marker='o', s=10, color='b', zorder=1)
plt.show()

<Figure size 720x720 with 1 Axes>

## Plot the temperature

In [53]:
TempRS = AS_DN.resample('M').mean()

In [54]:
T = np.array([griddata(AS_loc, np.array(TempRS)[i], (X, Y), method='cubic') for i in range(len(np.array(TempRS)))])
# TempRS

In [55]:
# T[8]
np.shape(X), np.shape(Y), np.shape(HSi), np.shape(AS_DN)

((693, 736), (693, 736), (84721, 28), (8408, 37))

In [34]:
# np.shape(np.array(TempRS)), np.shape(AS_loc), np.shape(EOFs), np.shape(T)
heatmap(Array=T[4], units=r'Temperature [$^\circ$C]', title='Temperature {0}'.format(TempRS.index[4]))

<Figure size 720x576 with 2 Axes>

In [35]:
# block = amp.blocks.Line(X, T)
# anim = amp.Animation([block])

# plt.show()

np.shape(X), np.shape(T)

((693, 736), (77, 693, 736))

In [36]:
# Voronoi Plots
# vor = Voronoi(AS_loc)
# heatmap(T[0], units=r'Temperature [$^\circ$C]', title='Temperature')
# plt.scatter(vor.vertices[:, 0], vor.vertices[:, 1])
# plt.show()

## Plot the empirical orthogonal functions 

In [37]:
EOFArray = AS.loc['01-May-2012':'31-Aug-2012']
EOFs, Evec, Eval = EOF(Array=EOFArray.resample('h').mean(), int_type='cubic')

In [38]:
hm1 = heatmap(EOFs[0], title=r'1st EOF of Vancouver Island using cubic interpolation. $\lambda=${0:.4f}'.format(Eval[0]), interp='none')
hm2 = heatmap(EOFs[1], title=r'2nd EOF of Vancouver Island using cubic interpolation. $\lambda=${0:.4f}'.format(Eval[1]), interp='none')
hm3 = heatmap(EOFs[2], title=r'3rd EOF of Vancouver Island using cubic interpolation. $\lambda=${0:.4f}'.format(Eval[2]), interp='none')
hm4 = heatmap(EOFs[3], title=r'4th EOF of Vancouver Island using cubic interpolation. $\lambda=${0:.4f}'.format(Eval[3]), interp='none')

<Figure size 720x576 with 2 Axes>

<Figure size 720x576 with 2 Axes>

<Figure size 720x576 with 2 Axes>

<Figure size 720x576 with 2 Axes>

In [39]:
plt.figure(figsize=(10, 4))
plt.plot(EOFArray.fillna(0)@Evec[0])
plt.tick_params(axis='both', which='major', labelsize=10, rotation=90)
# np.sqrt(Evec[0].dot(Evec[0]))

<Figure size 720x288 with 1 Axes>

## Welchy

In [40]:
Array = Mn_DN["DC"].loc['2015-06-01 00:00':'2015-09-30 23:59']

In [41]:
plot_power(Array, title='Power specturm with 95% confidence interval')

<Figure size 720x288 with 1 Axes>

## Wavelet transform

In [42]:
widths = np.arange(1, 31)
cwtmatr = cwt(np.array(AS_DN)[:, 0], ricker, widths)
plt.imshow(cwtmatr, cmap='bwr', aspect='auto',
            vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())
plt.yscale('symlog')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.title('Wavelet transform')

Text(0.5, 1.0, 'Wavelet transform')

<Figure size 432x288 with 1 Axes>

In [43]:
cwtmatr

array([[-0.22442817,  5.35351447,  2.41886332, ...,  1.25865616,
         6.55050855, 13.15358252],
       [-0.33787129,  5.13752165,  7.11305186, ..., 12.38876121,
        18.79324681, 15.76418878],
       [-0.80285644,  3.674221  ,  6.71174354, ..., 17.24222771,
        18.63514361, 14.31829873],
       ...,
       [ 8.19320945,  9.41797238, 10.59861001, ..., 10.05555487,
         6.60812836,  3.14907578],
       [ 7.83649962,  9.00598996, 10.13396666, ..., 10.25089018,
         6.86631449,  3.47004082],
       [ 7.44545624,  8.56386137,  9.64331642, ..., 10.43693056,
         7.11288024,  3.77710522]])

## Resample

In [44]:
RS = AS.resample('Y').mean().dropna(axis=0)

In [45]:
plt.figure(figsize=(10, 4))
RS.plot(figsize=(10, 4), linewidth=1)
# plt.scatter()
plt.legend('')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.show()

<Figure size 720x288 with 0 Axes>

<Figure size 720x288 with 1 Axes>

## Correlation between the different stations

In [46]:
MS.corr()

,DC,DE,HL,JB,JM,KT,US
DC,1.000000,0.983130,0.967003,0.950519,0.919920,0.987959,0.979110
DE,0.983130,1.000000,0.961855,0.939776,0.923591,0.978511,0.961845
HL,0.967003,0.961855,1.000000,0.946749,0.934992,0.966477,0.974121
JB,0.950519,0.939776,0.946749,1.000000,0.943190,0.952806,0.949653
JM,0.919920,0.923591,0.934992,0.943190,1.000000,0.925137,0.913019
KT,0.987959,0.978511,0.966477,0.952806,0.925137,1.000000,0.976123
US,0.979110,0.961845,0.974121,0.949653,0.913019,0.976123,1.000000


In [47]:
# AS.corr()